In [1]:
#pip install import_ipynb

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import random

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('data.json').read()
intents = json.loads(data_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [4]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)
print("model created")

371 documents
58 classes [' chancellor', 'Admission', 'B.Arch', 'BBA', 'BBA_fees', 'BCA', 'BCA_fees', 'BE_Block_location', 'BSC', 'Bachelor of Architecture (B.Arch)_fees', 'Btech_fees', 'COO', 'Canteen_location', 'Courses', 'E_Block_location', 'GJ_Block_location', 'G_Block_location', 'Gym_location', 'MBA', 'MBA_Block_location', 'MBA_fees', 'MCA', 'MCA_Department_location', 'MCA_fees', 'MRD_location', 'Master of Technology (MTech)_fees', 'PG_Courses', 'Ph.D.', 'Registrar', 'Research Programs_fees', 'UG_Courses', 'b.tech_m.tech', 'campus', 'clubs', 'elective', 'events', 'facilities', 'faculty', 'fees', 'food', 'goodbye', 'greeting', 'hostel', 'hours', 'infrastructure', 'library', 'library_location', 'location', 'metro', 'name', 'number', 'pro chancellor', 'ragging', 'random', 'swear', 'task', 'vacation', 'vice chancellor']
254 unique lemmatized words ['(', ')', '.', 'a', 'about', 'active', 'address', 'administration', 'admission', 'ajoy', 'all', 'am', 'antiragging', 'any', 'anyone', 'app

C:\Users\hp\AppData\Local\Temp\ipykernel_12628\1170111191.py:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


Epoch 1/200
75/75 [==============================] - 1s 2ms/step - loss: 4.0638 - accuracy: 0.0431
Epoch 2/200
75/75 [==============================] - 0s 2ms/step - loss: 3.9721 - accuracy: 0.0593
Epoch 3/200
75/75 [==============================] - 0s 2ms/step - loss: 3.9028 - accuracy: 0.0647
Epoch 4/200
75/75 [==============================] - 0s 2ms/step - loss: 3.7881 - accuracy: 0.1051
Epoch 5/200
75/75 [==============================] - 0s 2ms/step - loss: 3.7189 - accuracy: 0.1321
Epoch 6/200
75/75 [==============================] - 0s 2ms/step - loss: 3.5895 - accuracy: 0.1267
Epoch 7/200
75/75 [==============================] - 0s 2ms/step - loss: 3.4993 - accuracy: 0.1456
Epoch 8/200
75/75 [==============================] - 0s 2ms/step - loss: 3.3652 - accuracy: 0.1752
Epoch 9/200
75/75 [==============================] - 0s 2ms/step - loss: 3.1985 - accuracy: 0.2156
Epoch 10/200
75/75 [==============================] - 0s 1ms/step - loss: 3.1198 - accuracy: 0.2291
Epoch 11/

75/75 [==============================] - 0s 2ms/step - loss: 0.3868 - accuracy: 0.8787
Epoch 84/200
75/75 [==============================] - 0s 2ms/step - loss: 0.4167 - accuracy: 0.8383
Epoch 85/200
75/75 [==============================] - 0s 2ms/step - loss: 0.3523 - accuracy: 0.8949
Epoch 86/200
75/75 [==============================] - 0s 2ms/step - loss: 0.4266 - accuracy: 0.8518
Epoch 87/200
75/75 [==============================] - 0s 2ms/step - loss: 0.3689 - accuracy: 0.8949
Epoch 88/200
75/75 [==============================] - 0s 2ms/step - loss: 0.4069 - accuracy: 0.8760
Epoch 89/200
75/75 [==============================] - 0s 2ms/step - loss: 0.3759 - accuracy: 0.8841
Epoch 90/200
75/75 [==============================] - 0s 2ms/step - loss: 0.3658 - accuracy: 0.8868
Epoch 91/200
75/75 [==============================] - 0s 2ms/step - loss: 0.3667 - accuracy: 0.8787
Epoch 92/200
75/75 [==============================] - 0s 2ms/step - loss: 0.4033 - accuracy: 0.8760
Epoch 93/200


In [2]:
from keras.models import load_model
model = load_model('model.h5')
import json
import random
intents = json.loads(open('data.json').read())
words = pickle.load(open('texts.pkl','rb'))
classes = pickle.load(open('labels.pkl','rb'))

In [ ]:

def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

from flask import Flask, render_template, request

app = Flask(__name__)

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    return chatbot_response(userText)

if __name__ == "__main__":
    #app.debug = True
    app.run(port=3053)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3053
Press CTRL+C to quit
